## Question

A transformation sequence from word `beginWord` to word `endWord` using a dictionary `wordList` is a sequence of words `beginWord -> s1 -> s2 -> ... -> sk` such that:

    Every adjacent pair of words differs by a single letter.
    Every si for 1 <= i <= k is in wordList. Note that beginWord does not need to be in wordList.
    sk == endWord

Given two words, `beginWord` and `endWord`, and a dictionary `wordList`, return the number of words in the shortest transformation sequence from `beginWord` to `endWord`, or 0 if no such sequence exists.


Example 1:

Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]

Output: 5

Explanation: One shortest transformation sequence is "hit" -> "hot" -> "dot" -> "dog" -> cog", which is 5 words long.

Example 2:

Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"]

Output: 0

Explanation: The endWord "cog" is not in wordList, therefore there is no valid transformation sequence.

 

Constraints:

    1 <= beginWord.length <= 10
    endWord.length == beginWord.length
    1 <= wordList.length <= 5000
    wordList[i].length == beginWord.length
    beginWord, endWord, and wordList[i] consist of lowercase English letters.
    beginWord != endWord
    All the words in wordList are unique.



## Solution

First, create an adjacency mapping for each word in the `wordList`. What that means is, for each word, what are other words in the wordList that it can go with one character swapping. If the word is "dog", 
There are three places where characters can be swapped

1) First character -> (*)og
2) Second character -> d(*)g
3) Third character -> do(*)

Our adjacency mapping maps these patterns to words in the `wordList`

For exapmle, if we have, `beginWord` dig, `endWord` doc and `wordList` [dog, cat, dig, cot, hog, dot, doc, rat, bat, cap, car], the adjacency mapping will be:
       
       {"*og": ['dog', 'hog'],
         "d*g": ['dog', 'dig'],
         "do*": ['dog', 'dot', 'doc'],
         "*at": ['cat', 'rat', 'bat'],
         "c*t": ['cat', 'cot'],          
         "ca*": ['cat', 'cap', 'car'],
         "*ig": ['dig'],
         "di*": ['dig'],
         "*ot": ['cot', 'dot'],
         "co*": ['cot'],
         "h*g": ['hog'],
         "ho*": ['hog'],
         "d*t": ['dot'],
         "*oc": ['doc'],
         "d*c": ['doc'],
         "r*t": ['rat'],
         "ra*": ['rat'],
         "b*t": ['bat'],
         "ba*": ['bat'],
         "*ap": ['cap'],
         "c*p": ['cap'],
         "*ar": ['car'],
         "c*r": ['car']
        }
 
Having this mapping, we just need to check whether a character for the current word can be swapped to get to `endWord`. So, we try to swap every possible character position in the current word and look up the value in our adjacency mapping. We do that for each word in the adjacency mapping until we find the `endWord`. If we encounter `endWord` while checking for words in adjacency mapping, we can return the 1+level at which we are in. If we have exhausted everything, it is sure that we have not found the word and we can return 0.

A visual progression of search using this adajacency mapping is as follows:

<img src='assets/word_ladder/visual.png'> 

In [15]:
from collections import deque, defaultdict
from typing import List

def ladderLength(beginWord: str, endWord: str, wordList: List[str]) -> int:
    if endWord not in wordList:  # O(n)
        return 0
    if endWord == beginWord:
        return 1

    L = len(beginWord)

    # Building adjacency mapping
    # For every word we encounter, we will swap character at every position of the word
    # and look up possible words we can form from the current word
    all_combo_dict = defaultdict(list)
    for word in wordList:  # O(n)
        for i in range(L):  # O(m)
            all_combo_dict[word[:i] + "*" + word[i + 1 :]].append(
                word
            )  # Substring operation O(m)
    # Overall runtime: O(m^2 * n)

    level = 1
    # Push to BFS queue with beginWord so that we can start the ball rolling
    queue = deque([(beginWord, level)])

    # Push words which we can encountered to hash set so that we don't need to process
    # the word which we have already seen we can have this luxury since we are doing BFS 
    # and if we are looking at the previouly looked word, we are sure that it cannot lead
    # to a shorter path than then path which we have taken before
    seen_words = set([beginWord])

    while queue:  # This will be exhausted when we have completed our BFS tree
        currentWord, level = queue.popleft()
        for i in range(L): # O(m)
            # For each character index of the currentWord, we want to check
            # whether one of the adjacent words is the end word
            # So, we iterate through all adjacent words to check for that
            # If an adjacent word is not the end word, we will continue doing the same for
            # words mapped in adjacency mapping. Therefore, we push to the queue so they will
            # be processed afterwards

            intermediate_word = currentWord[:i] + "*" + currentWord[i + 1 :] # O(m)
            for word in all_combo_dict[intermediate_word]: # O(n)
                if word == endWord:
                    return level + 1
                if word not in seen_words:
                    queue.append((word, level + 1))
                    seen_words.add(word)

    return 0

Runtime: O(m^2 n); where m is length of word and n is the length of word list

Space: O(m^2 n)

In [18]:
# test cases for the problem
# [(input grid, number of paths to get to goal for that grid)]

test_cases = [
    ({"beginWord": "hit", "endWord": "cog", "wordList": ["hot","dot","dog","lot","log","cog"]}, 5),
    ({"beginWord": "hit", "endWord": "cog", "wordList": ["hot","dot","dog","lot","log"]}, 0)
]

In [19]:
for tc in test_cases:
    question, answer = tc
    result = ladderLength(**question)
    assert (
        result == answer
    ), f"Failed for case {question}. Expected {answer}; got back {result}"
    print("Passed")

Passed
Passed
